# <center>Композиции алгоритмов

Будем решать задачу кредитного скоринга.

#### Данные по кредитному скорингу представлены следующим образом:

##### Прогнозируемая  переменная
* SeriousDlqin2yrs	– наличие длительных просрочек выплат платежей за 2 года.

##### Независимые признаки
* age – возраст заёмщика (число полных лет);
* NumberOfTime30-59DaysPastDueNotWorse	– количество раз, когда заёмщик имел просрочку выплаты других кредитов 30-59 дней в течение последних двух лет;
* NumberOfTime60-89DaysPastDueNotWorse – количество раз, когда заёмщик имел просрочку выплаты других кредитов 60-89 дней в течение последних двух лет;
* NumberOfTimes90DaysLate – количество раз, когда заёмщик имел просрочку выплаты других кредитов более 90 дней;
* DebtRatio – ежемесячные отчисления на задолжености (кредиты, алименты и т.д.) / совокупный месячный доход;
* MonthlyIncome	– месячный доход в долларах;
* NumberOfDependents – число человек в семье кредитозаёмщика.

In [1]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier

In [2]:
def impute_nan_with_median(table):
    for col in table.columns:
        table[col]= table[col].fillna(table[col].median())
    return table

In [3]:
data = pd.read_csv('credit_scoring_sample.csv', sep=";")
data.head()

,SeriousDlqin2yrs,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,NumberOfTimes90DaysLate,NumberOfTime60-89DaysPastDueNotWorse,MonthlyIncome,NumberOfDependents
0,0,64,0,0.249908,0,0,8158.0,0.0
1,0,58,0,3870.000000,0,0,NaN,0.0
2,0,41,0,0.456127,0,0,6666.0,0.0
3,0,43,0,0.000190,0,0,10500.0,2.0
4,1,49,0,0.271820,0,0,400.0,0.0


In [4]:
independent_columns_names = data.columns.values
independent_columns_names = [x for x in data if x != 'SeriousDlqin2yrs']
independent_columns_names

['age',
 'NumberOfTime30-59DaysPastDueNotWorse',
 'DebtRatio',
 'NumberOfTimes90DaysLate',
 'NumberOfTime60-89DaysPastDueNotWorse',
 'MonthlyIncome',
 'NumberOfDependents']

In [5]:
table = impute_nan_with_median(data)

In [6]:
X = table[independent_columns_names]
y = table['SeriousDlqin2yrs']
X.shape

(45063, 7)

In [7]:
y.value_counts()

0    35037
1    10026
Name: SeriousDlqin2yrs, dtype: int64

Задайте решающее дерево, пользуясь встроенной функцией `DecisionTreeClassifier` с параметрами `random_state=17` и `class_weight='balanced'`.

In [8]:
clf = DecisionTreeClassifier(class_weight='balanced',random_state=17)
clf.fit(X, y)

DecisionTreeClassifier(class_weight='balanced', random_state=17)

Используйте функцию `GridSearchCV` для выбора оптимального набора гиперпараметров для указанной задачи. В качестве метрики качества возьмите ROC AUC.

In [9]:
max_depth_values = [3, 5, 6, 7, 9]
max_features_values = [4, 5, 6, 7]
tree_params = {'max_depth': max_depth_values,
               'max_features': max_features_values}

In [10]:
gscv = GridSearchCV(clf,tree_params,scoring ='roc_auc')
gscv.fit(X, y)

GridSearchCV(estimator=DecisionTreeClassifier(class_weight='balanced',
                                              random_state=17),
             param_grid={'max_depth': [3, 5, 6, 7, 9],
                         'max_features': [4, 5, 6, 7]},
             scoring='roc_auc')

In [11]:
gscv.best_params_

{'max_depth': 6, 'max_features': 7}

Зафиксируйте кросс-валидацию с помощью функции `StratifiedKFold` на 5 разбиений с перемешиванием, `random_state=17`.

In [12]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

In [13]:
scores=cross_val_score(gscv,X,y,cv=skf,scoring ='roc_auc')
scores

array([0.8199366 , 0.81925313, 0.81982122, 0.81513823, 0.81655099])

Какое максимальное значение ROC AUC получилось?

In [14]:
np.max(scores)

0.8199366025575331

# Реализация случайного леса

Теперь реализуйте случайный лес. В качестве базового алгоритма здесь всегда выступает `DecisionTreeClassifier`.

На каждом шаге алгоритма необходимо запоминать индексы признаков, которые участвовали в обучении леса.

- В методе `fit` в цикле (`i` от 0 до `n_estimators-1`) фиксируйте seed, равный (`random_state + i`). Это нужно для того, чтобы на каждой итерации seed был новый, при этом все значения можно было бы воспроизвести.
- Зафиксировав seed, сделайте bootstrap-выборку (т.е. с замещением) из множества id объектов. Размер bootstrap-выборки = размеру исходной.
- Зафиксировав seed, выберите **без замещения** `max_features` признаков, сохраните список выбранных id признаков в `self.feat_ids_by_tree`.
- Обучите дерево с теми же `max_depth`, `max_features` и `random_state`, что и у `RandomForestClassifierCustom` на выборке с нужным подмножеством объектов и признаков.
- В методе `predict_proba` у тестовой выборки нужно взять те признаки, на которых соответствующее дерево обучалось, и сделать прогноз вероятностей (`predict_proba` уже для дерева, вернуть вероятности класса 1). Метод должен вернуть усреднение прогнозов по всем деревьям.

In [15]:
from sklearn.base import BaseEstimator

class RandomForestClassifierCustom(BaseEstimator):
    def __init__(self, n_estimators=10, max_depth=3, max_features=10, random_state=17):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.max_features = max_features
        self.random_state = random_state
        # в данном списке будем хранить отдельные деревья
        self.trees = []
        # тут будем хранить списки индексов признаков, на которых обучалось каждое дерево 
        self.feat_ids_by_tree = []
        
    def fit(self, X, y):
        for i in range(self.n_estimators):
            
            np.random.seed(self.random_state+i) #фиксация seed
            # bootstrap-выборка (т.е. с замещением) из множества id объектов.
            btsr_ids = np.unique((np.random.choice(range(X.shape[0]),X.shape[0]))) 
            #выбрать без замещения max_features признаков
            feat_ids = np.random.choice(range(X.shape[1]),self.max_features,replace=False)
            #сохранить список выбранных id признаков в self.feat_ids_by_tree
            self.feat_ids_by_tree.append(feat_ids)
            
            clf = DecisionTreeClassifier(class_weight='balanced',random_state = self.random_state,
                                        max_depth=self.max_depth, max_features=self.max_features)
            
            clf.fit(X[btsr_ids, :][:, feat_ids], y[btsr_ids])
            self.trees.append(clf)
        return self
        
    def predict_proba(self, X):
        pred = []
        for i in range(self.n_estimators):
            ids = self.feat_ids_by_tree[i]
            pred.append(self.trees[i].predict_proba(X[:,ids]))
        return np.mean(pred, axis=0)
    
    def decision_function(self, X):
        res = self.predict_proba(X)
        return np.array([x[1] for x in res])

Проведите кросс-валидацию. Какое получилось среднее значение ROC AUC на кросс-валидации? Сравните качество вашей реализации с реализацией `RandomForestClassifier` из `sklearn`. Аналогично предыдущему заданию, подберите гиперпараметры для случайного леса.

In [16]:
model= RandomForestClassifierCustom(max_depth=7, max_features=6)
model.fit(X.values, y.values)

RandomForestClassifierCustom(max_depth=7, max_features=6)

In [17]:
rfcc_scores = cross_val_score(RandomForestClassifierCustom(max_depth=7, max_features=6), 
                          X.values, y.values, scoring="roc_auc", cv=skf)
rfcc_scores


array([0.8341511 , 0.83519042, 0.83297176, 0.82787647, 0.8290295 ])

Какое получилось среднее значение ROC AUC на кросс-валидации? 

In [18]:
np.mean(rfcc_scores)

0.8318438503021394

 Сравните качество вашей реализации с реализацией RandomForestClassifier из sklearn

In [19]:
rfc_scores = cross_val_score(RandomForestClassifier(max_depth=7, max_features=6), 
                          X.values, y.values, scoring="roc_auc", cv=skf)
rfc_scores

array([0.83408815, 0.83434034, 0.83219188, 0.82911378, 0.83039956])

In [20]:
np.mean(rfc_scores)

0.8320267425111535

Реализация sklearn дает немного лучше результат, но ручная реализация показывает себя достойно.

Аналогично предыдущему заданию, подберите гиперпараметры для случайного леса.

In [21]:
clf = RandomForestClassifier(class_weight='balanced',random_state=17)
clf.fit(X, y)

RandomForestClassifier(class_weight='balanced', random_state=17)

In [22]:
max_depth_values = [3, 5, 6, 7, 9]
max_features_values = [4, 5, 6, 7]
tree_params = {'max_depth': max_depth_values,
               'max_features': max_features_values}

In [23]:
gscv = GridSearchCV(clf,tree_params,scoring ='roc_auc')
gscv.fit(X, y)

GridSearchCV(estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=17),
             param_grid={'max_depth': [3, 5, 6, 7, 9],
                         'max_features': [4, 5, 6, 7]},
             scoring='roc_auc')

In [24]:
gscv.best_params_

{'max_depth': 9, 'max_features': 4}